In [7]:
import re
import string
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_col_to_name

In [8]:
def randStr(length):
    rand_str = "".join(np.random.choice(np.array(list(string.ascii_uppercase)),length))
    return rand_str

In [9]:
num = 7
column_names = [randStr(5) for i in range(num)]

my_df = pd.DataFrame(np.random.rand(num,num))
my_df.columns = column_names

In [10]:
my_df

,OSMZU,RFIZD,MJICJ,LDGJH,TXJMV,MOOCL,QIJAX
0,0.707801,0.431912,0.311527,0.749288,0.176746,0.661225,0.171821
1,0.856206,0.421802,0.701306,0.357513,0.488640,0.878906,0.310176
2,0.441597,0.579130,0.590644,0.355610,0.760915,0.836585,0.164845
3,0.850957,0.510595,0.482433,0.206053,0.178119,0.852826,0.660721
4,0.632590,0.845504,0.232336,0.578246,0.279180,0.290978,0.196799
5,0.852716,0.004518,0.287152,0.119131,0.496080,0.400799,0.825146
6,0.960913,0.145776,0.514915,0.427011,0.891234,0.394059,0.481943


In [11]:
def spanXL(dataframe, start_index=0, row = 1):
    first = chr(65+start_index)
    last = xl_col_to_name(dataframe.shape[1]-1)
    span = [first+str(row),last+str(dataframe.shape[1]+1)]
    span = ":".join(span)
    return span
spanXL(my_df)

'A1:G8'

In [12]:
def colSpanXL(dataframe, start_index=0, row = 1):
    span_str = spanXL(dataframe, start_index, row).replace(":","&")
#     span_str = re.sub("\d+$","1",span_str)
    rx = re.compile("(\d.+\D)")
    col_span = "$".join(rx.split(span_str))
    return col_span

colSpanXL(my_df)

'A$1&G$8'

In [13]:
def makeSearch(term,df):
    search_str = '=SEARCH("{0}",{1})'.format(term,colSpanXL(df))
    return search_str

In [14]:
makeSearch("T",my_df)

'=SEARCH("T",A$1&G$8)'

In [15]:
def dynFormat(format_set,workbook):
    return workbook.add_format(format_set)

In [17]:

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_conditional.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
my_df.to_excel(writer, sheet_name='Sheet1',index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']


worksheet.conditional_format(spanXL(my_df), {'type':'formula',
                                             'criteria': makeSearch(column_names[3],my_df),
                                             'format': dynFormat({"bg_color":"red"},workbook)})

worksheet.conditional_format(spanXL(my_df),{'type':'formula',
                                             'criteria': makeSearch(column_names[-1],my_df),
                                             'format':  dynFormat({"bg_color":"#CCFFCC"},workbook)})

# Close the Pandas Excel writer and output the Excel file.
writer.save()